 ## Task 4：源大模型微调实战

### 2.1 环境准备

In [1]:
# 查看已安装依赖
# ! pip list

In [2]:
# 安装 streamlit
!pip install streamlit==1.24.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### 2.2 模型下载

In [3]:
from modelscope import snapshot_download
model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='.')

### 2.3 数据处理

In [4]:
# 导入环境

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from modelscope.msdatasets import MsDataset
from datasets import Dataset
import pandas as pd


2024-08-25 23:12:52.470066: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 23:12:52.481427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 23:12:52.494895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 23:12:52.498957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 23:12:52.508999: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# 读取数据
df = pd.read_json('./data1.json')
ds = Dataset.from_pandas(df)
# 加载 MsDataset 数据集
# train_datasets = MsDataset.load('multico_ner_2023_wiki128', subset_name='zh', namespace='pangda', split='train')
# dev_datasets = MsDataset.load('multico_ner_2023_wiki128', subset_name='zh', namespace='pangda', split='dev')
# test_datasets = MsDataset.load('multico_ner_2023_wiki128', subset_name='zh', namespace='pangda', split='test')

# # 提取 MsDataset 中的数据
# def extract_data(msdataset):
#     data = []
#     for sample in msdataset:
#         data.append(sample)
#     return data

# # 将提取的数据转换为 pandas DataFrame
# train_data = extract_data(train_datasets)
# dev_data = extract_data(dev_datasets)
# test_data = extract_data(test_datasets)

# train_df = pd.DataFrame(train_data)
# dev_df = pd.DataFrame(dev_data)
# test_df = pd.DataFrame(test_data)

# # 将 pandas DataFrame 转换为 datasets.Dataset
# train_ds = Dataset.from_pandas(train_df)
# dev_ds = Dataset.from_pandas(dev_df)
# test_ds = Dataset.from_pandas(test_df)

# # 选择一个数据集用于示例（例如 train_ds）
# ds = dev_ds

In [6]:
# 查看数据
len(ds)
ds[:1]
# 查看数据集中一个样本的字段名
# print(train_ds[0])


{'input': ['现在18岁了，最近半年，发觉，性生活总是提不起劲，同时，每次才开始就已经射了，请问：男孩早泄究竟是什么因素引发的。'],
 'output': ['可能诱发早泄的因素，与多个方面都有一定的影响，常见的是患者自身的心里因素，在性生活时受到一些不良因素的影响，或者是夫妻感情不合，也可能是因为患者自身缺少性知识，常常手淫就会引起早泄，除了这两点，男性如果患上一些男科疾病也会诱发早泄，比如像前列腺炎、前列腺钙化等，或是骨盆骨折以及高血糖等，早泄对男性身心造成的创伤都是较大的，建议了解病因后正确预防。']}

In [7]:
# 加载 tokenizer
path = './IEITYuan/Yuan2-2B-Mars-hf'

tokenizer = AutoTokenizer.from_pretrained(path, add_eos_token=False, add_bos_token=False, eos_token='<eod>')
tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)
tokenizer.pad_token = tokenizer.eos_token

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [8]:
# 定义数据处理函数
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

    instruction = tokenizer(f"{example['input']}<sep>")
    response = tokenizer(f"{example['output']}<eod>")
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = [1] * len(input_ids) 
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] # instruction 不计算loss

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# def process_func(example):
#     MAX_LENGTH = 384    # 设定最大长度

#     # 将 tokens 拼接为字符串
#     tokens = example['tokens']
#     spans = example['spans']
    
#     # 将 tokens 转换为模型输入 ID
#     encodings = tokenizer(tokens, is_split_into_words=True, truncation=True, padding='max_length', max_length=MAX_LENGTH)
#     input_ids = encodings["input_ids"]
#     attention_mask = encodings["attention_mask"]
    
#     # 创建标签数组
#     labels = [-100] * len(input_ids)  # 初始化所有标签为 -100
    
#     # 将 spans 中的实体标签对齐到 token ID 上
#     for span in spans:
#         start, end = span['start'], span['end']
#         label = span['type']
#         # 在这里处理对齐标签的逻辑
#         # 请根据实际情况对齐标签，下面的示例是简化版本
#         for idx in range(start, end + 1):
#             if idx < len(labels):  # 防止索引超出范围
#                 labels[idx] = label_to_id(label)  # 将实体标签转换为 ID
    
#     return {
#         "input_ids": input_ids,
#         "attention_mask": attention_mask,
#         "labels": labels
#     }

# def label_to_id(label):
#     # 假设有一个标签到 ID 的映射字典
#     label2id = {"Politician": 1, "Location": 2, "Organization": 3, "Date": 4}
#     return label2id.get(label, -100)  # 默认标签 -100


In [9]:
# 处理数据集
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 828
})

In [10]:
# 数据检查
tokenizer.decode(tokenized_id[0]['input_ids'])

'现在18岁了，最近半年，发觉，性生活总是提不起劲，同时，每次才开始就已经射了，请问：男孩早泄究竟是什么因素引发的。<sep> 可能诱发早泄的因素，与多个方面都有一定的影响，常见的是患者自身的心里因素，在性生活时受到一些不良因素的影响，或者是夫妻感情不合，也可能是因为患者自身缺少性知识，常常手淫就会引起早泄，除了这两点，男性如果患上一些男科疾病也会诱发早泄，比如像前列腺炎、前列腺钙化等，或是骨盆骨折以及高血糖等，早泄对男性身心造成的创伤都是较大的，建议了解病因后正确预防。<eod>'

In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'可能诱发早泄的因素，与多个方面都有一定的影响，常见的是患者自身的心里因素，在性生活时受到一些不良因素的影响，或者是夫妻感情不合，也可能是因为患者自身缺少性知识，常常手淫就会引起早泄，除了这两点，男性如果患上一些男科疾病也会诱发早泄，比如像前列腺炎、前列腺钙化等，或是骨盆骨折以及高血糖等，早泄对男性身心造成的创伤都是较大的，建议了解病因后正确预防。<eod>'

### 2.4 模型训练

In [12]:
# 模型加载
model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model

YuanForCausalLM(
  (model): YuanModel(
    (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
    (layers): ModuleList(
      (0-23): 24 x YuanDecoderLayer(
        (self_attn): YuanAttention(
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): YuanRotaryEmbedding()
          (lf_gate): LocalizedFiltering(
            (conv1): Conv2d(2048, 1024, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (conv2): Conv2d(1024, 2048, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (output_layernorm): YuanRMSNorm()
          )
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): YuanMLP(
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (gate_proj): Linear(in_features=2048, out_featu

In [13]:
model.enable_input_require_grads() # 开启gradient_checkpointing时，要执行该方法

In [14]:
# 查看模型数据类型
model.dtype

torch.bfloat16

In [15]:
# 配置Lora
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'k_proj', 'q_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [16]:
# 构建PeftModel
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): YuanForCausalLM(
      (model): YuanModel(
        (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
        (layers): ModuleList(
          (0-23): 24 x YuanDecoderLayer(
            (self_attn): YuanAttention(
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (o_proj): lora.Linear(
                (base_layer): Linear(in_

In [17]:
# 打印需要训练的参数
model.print_trainable_parameters()

trainable params: 9,043,968 || all params: 2,097,768,448 || trainable%: 0.4311


In [18]:
# 设置训练参数
args = TrainingArguments(
    output_dir="./output/Yuan2.0-2B_lora_bf16",
    per_device_train_batch_size=12,
    gradient_accumulation_steps=1,
    logging_steps=1,
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=5e-5,
    save_on_each_node=True,
    gradient_checkpointing=True,
    bf16=True
)

In [19]:
# 初始化Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
# 模型训练
trainer.train()

[2024-08-25 23:13:02,706] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,5.835100
2,5.671700
3,5.451400
4,4.973200
5,4.637200
6,4.488100
7,3.944700
8,4.145900
9,3.889700
10,3.702600


/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./IEITYuan/Yuan2-2B-Mars-hf - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./IEITYuan/Yuan2-2B-Mars-hf - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the

TrainOutput(global_step=207, training_loss=2.058268141631343, metrics={'train_runtime': 226.2435, 'train_samples_per_second': 10.979, 'train_steps_per_second': 0.915, 'total_flos': 5028584338243584.0, 'train_loss': 2.058268141631343, 'epoch': 3.0})

### 2.5 效果验证

In [26]:
# 定义生成函数
def generate(prompt):
    prompt = prompt + "<sep>"
    inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].cuda()
    outputs = model.generate(inputs, do_sample=False, max_length=812)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(output.split("<sep>")[-1])


In [28]:
# 输入prompt template
# template = '''
# # 任务描述
# 假设你是一个AI助手，能从简历中识别出所有的命名实体，并以json格式返回结果。

# # 任务要求
# 实体的类别包括：姓名、国籍、种族、职位、教育背景、专业、组织名、地名。
# 返回的json格式是一个字典，其中每个键是实体的类别，值是一个列表，包含实体的文本。

# # 样例
# 输入：
# 张三，男，中国籍，工程师
# 输出：
# {"姓名": ["张三"], "国籍": ["中国"], "职位": ["工程师"]}

# # 当前简历
# input_str

# # 任务重述
# 请参考样例，按照任务要求，识别出当前简历中所有的命名实体，并以json格式返回结果。
# '''

# 输入prompt template
# template = '''
# # 任务描述
# 假设你是一个AI助手，能够从给定文本中识别出所有的命名实体，并以JSON格式返回结果。

# # 任务要求
# 实体的类别包括：姓名、组织名、地名、职位、日期、其他。
# 返回的JSON格式是一个字典，其中每个键是实体的类别，值是一个列表，包含该类别下的所有实体文本。

# # 样例
# 输入：
# 张三是北京大学的教授。李四于2024年在上海获得了硕士学位。
# 输出：
# {"姓名": ["张三", "李四"], "地名": ["北京", "上海"], "职位": ["教授"], "日期": ["2024年"]}

# # 当前文本
# input_str

# # 任务重述
# 请参考样例，按照任务要求，识别出当前文本中所有的命名实体，并以JSON格式返回结果。
# '''
# # 任务描述
# 假设你是一个AI医疗诊断助手，能够根据用户提供的症状描述给出相应的医疗建议。

# # 任务要求
# 返回的医疗建议应该包括诊断、治疗建议、可能的原因以及预防措施。# 样例
# 输入：
# 男宝，刚满3个月，一开始，说话时觉得嗓子疼，发现，咳嗽好像也比较严重，并且，一直都有点发烧，请问：小孩扁桃体炎难受怎样治效果才好?
# 输出：
# 在治疗上的话一般建议遵医嘱给孩子吃点消炎的药物，除全身用药的话也可进行局部治疗，比如扁桃体隐窝冲洗，或是扁桃体内药物注射等，必要的时候，比如孩子咽痛尤甚的时候，也是可以给点止痛药的，如果孩子还有点发烧的话那么就应该酌情进行退烧，要是高于40摄氏度的话，那么就需要尽早就医了，除了这些药物治疗的方法之外，如果孩子总是反复发生炎症的话，家长也可在医生的见一下选择手术进行治疗的，期间注意让孩子充分休息，并且多吃一些蔬菜水果。

template = '''



# 当前文本
input_str

# 任务重述
请参考样例，按照任务要求，根据当前文本中的症状描述给出医疗建议。
'''



In [29]:
input_str = '女宝宝，才1个月，一开始，说嗓子喝水都觉得疼，发现，吞咽好像有点困难，并且，好像低烧一直都没退，请问：小孩扁桃体炎难受怎样治疗?'
prompt = template.replace('input_str', input_str).strip()
generate(prompt)

# 当前文本
女宝宝，才1个月，一开始，说嗓子喝水都觉得疼，发现，吞咽好像有点困难，并且，好像低烧一直都没退，请问：小孩扁桃体炎难受怎样治疗?

# 任务重述
请参考样例，按照任务要求，根据当前文本中的症状描述给出医疗建议。 根据你的描述，孩子扁桃体炎的症状主要是咽痛、发烧、咳嗽，并且孩子还伴发疼痛的症状，建议家长是可以酌情给点镇痛药缓解的，如果孩子还有点发烧的话那么就应该酌情进行退烧，如果高于40摄氏度的话，那么就需要尽早就医了，扁桃体炎是具有反复发作的特征的，如果存在类似情况，炎症得到控制后可以考虑手术，治疗期间还要注意保持饮食卫生，以及足够的休息时间。
